In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np
import urllib
from time import sleep

In [2]:
# define a function to get the soup
def get_soup(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    return soup

# define a function to increment the url to the next page

def next_page(base, player_count):
    url = base+str(player_count)
    return url

# define a function that will return a list of the rows of player stats on a page
def get_rows(soup):
    body = soup.findAll('tbody')
    rows = body[0].findAll('tr')
    stat_rows = [x for x in rows if x['class']==['']]
    return stat_rows

# define a function that will strip the stats out of a row of player html data
def strip_stats(rows):
    for x in range(0,len(rows)):
        player_stats = rows[x].findAll('td')
        stats.append([z.text for z in player_stats])

In [3]:
base = ('http://www.pro-football-reference.com/play-index/psl_finder.cgi?request=1&match=single&year_min=1999&year_max=2016&season_start=1&season_end=-1&age_min=0&age_max=99&league_id=NFL&team_id=&is_active=&is_hof=&pos_is_lb=Y&c1stat=sacks&c1comp=gt&c1val=&c2stat=fumbles_rec_yds&c2comp=gt&c2val=&c3stat=def_int_yds&c3comp=gt&c3val=&c4stat=seasons&c4comp=gt&c4val=&c5comp=height_in&c5gtlt=eq&c6mult=1.0&c6comp=&order_by=g&draft=0&draft_year_min=1936&draft_year_max=2016&type=&draft_round_min=0&draft_round_max=99&draft_slot_min=1&draft_slot_max=500&draft_pick_in_round=0&draft_league_id=&draft_team_id=&college_id=all&conference=any&draft_pos_is_qb=Y&draft_pos_is_rb=Y&draft_pos_is_wr=Y&draft_pos_is_te=Y&draft_pos_is_e=Y&draft_pos_is_t=Y&draft_pos_is_g=Y&draft_pos_is_c=Y&draft_pos_is_ol=Y&draft_pos_is_dt=Y&draft_pos_is_de=Y&draft_pos_is_dl=Y&draft_pos_is_ilb=Y&draft_pos_is_olb=Y&draft_pos_is_lb=Y&draft_pos_is_cb=Y&draft_pos_is_s=Y&draft_pos_is_db=Y&draft_pos_is_k=Y&draft_pos_is_p=Y&offset=')

# The master function will need to have a stats and counter that increments by 100
# and prints out how many players it has processed per loop
def get_lb_stats(base_url, limit):
    player_count = 0
    while player_count <= limit:
        url = next_page(base, player_count)
        soup = get_soup(url)
        rows = get_rows(soup)
        strip_stats(rows)
        player_count +=100
        print '%d players processed' % player_count
        sleep(.5)

In [4]:
# instantiate the list that the player stats will be added to. I've yet to figure
# out how to have this list exist inside the function and be returned by it.
stats = []

In [6]:
# after looking through the results of the search on pro-football-reference, this
# upper limit will capture all of the players in the search results.
get_lb_stats(base, 4600)

100 players processed
200 players processed
300 players processed
400 players processed
500 players processed
600 players processed
700 players processed
800 players processed
900 players processed
1000 players processed
1100 players processed
1200 players processed
1300 players processed
1400 players processed
1500 players processed
1600 players processed
1700 players processed
1800 players processed
1900 players processed
2000 players processed
2100 players processed
2200 players processed
2300 players processed
2400 players processed
2500 players processed
2600 players processed
2700 players processed
2800 players processed
2900 players processed
3000 players processed
3100 players processed
3200 players processed
3300 players processed
3400 players processed
3500 players processed
3600 players processed
3700 players processed
3800 players processed
3900 players processed
4000 players processed
4100 players processed
4200 players processed
4300 players processed
4400 players process

In [7]:
# running a len on the stats column shows that we got all of the players we wanted
print len(stats)
# manually create the column names that we'll need for this dataframe
lb_cols = ['rk', 'name', 'year', 'age', 'drafted', 'team', 'league',
            'height', 'weight', 'bmi', 'games', 'starts', 'sacks', 'tkls',
           'ast_tkls', 'ints', 'int_return_yds', 'int_return_tds',
           'passes_defended', 'times_fumbled', 'fumb_rec', 'yds_fumb_rec',
           'td_fumb_rec', 'forced_fumbles', 'yrs', 'pro_bowl', 'all_pro', 'av']
df = pd.DataFrame(stats, columns = lb_cols)

4634


In [8]:
## checking the tail since we may have looped through an extra page
df.tail()

,rk,name,year,age,drafted,team,league,height,weight,bmi,...,passes_defended,times_fumbled,fumb_rec,yds_fumb_rec,td_fumb_rec,forced_fumbles,yrs,pro_bowl,all_pro,av
4629,4630,J.K. Schaffer,2012,22,,CIN,NFL,6-0,232,31.5,...,0,0,0,0,0,0,1,0,0,0
4630,4631,Pat Schiller,2012,24,,ATL,NFL,6-2,235,30.2,...,0,0,0,0,0,0,1,0,0,0
4631,4632,Brian Smith,2012,23,,BUF,NFL,6-3,240,30.0,...,0,0,0,0,0,0,1,0,0,0
4632,4633,Sean Spence,2012,23,3-86,PIT,NFL,5-11,231,32.2,...,0,0,0,0,0,0,1,0,0,0
4633,4634,Jabara Williams,2012,23,7-228,STL,NFL,6-2,220,28.2,...,0,0,0,0,0,0,1,0,0,0


In [9]:
## checking to make sure we looped through all the pages by seeing 
## the number of unique names in the name column
df.name.nunique()

1341

In [10]:
# identify the numeric columns so that they can be converted before adding to sql
numeric_columns = ['rk', 'year', 'age', 'weight', 'bmi', 'games', 'starts',
                   'sacks', 'tkls','ast_tkls', 'ints', 'int_return_yds',
                   'int_return_tds', 'passes_defended', 'times_fumbled',
                   'fumb_rec', 'yds_fumb_rec', 'td_fumb_rec', 'forced_fumbles', 
                   'yrs', 'pro_bowl', 'all_pro', 'av']
                  

# this loop will force the numeric columns to integers and floats depending on their nature
for col in numeric_columns:
    df[col] = df[col].convert_objects(convert_numeric=True)

df.dtypes

/Users/TerryONeill/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:11: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


rk                   int64
name                object
year                 int64
age                float64
drafted             object
team                object
league              object
height              object
weight             float64
bmi                float64
games              float64
starts             float64
sacks              float64
tkls                 int64
ast_tkls             int64
ints                 int64
int_return_yds       int64
int_return_tds       int64
passes_defended      int64
times_fumbled        int64
fumb_rec             int64
yds_fumb_rec         int64
td_fumb_rec          int64
forced_fumbles       int64
yrs                  int64
pro_bowl             int64
all_pro              int64
av                   int64
dtype: object

In [11]:
## Creating a csv of the newly formed qb database

df.to_csv('lb_stats', encoding = 'utf-8')

In [12]:
## I will now be creating a database in postgres in order to add
## this dataframe as a table to perform queries on outside of python

from sqlalchemy import create_engine
import psycopg2

engine = create_engine('postgresql://TerryONeill@localhost:5432/nfl_capstone')

In [13]:
## this is adding the dataframe to my newly created database in psql as
## a table named 'running_back_stats'
df.to_sql('lb_stats', engine)